In [9]:
# !pip install pyproj
# !pip install geopandas
# !pip install vincent

In [10]:
import pathlib
import random
from functools import reduce
from collections import defaultdict
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re
from matplotlib import font_manager, rc, rcParams
from shapely.geometry import Point as shapely_Point
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi
from IPython.display import display
import branca
from branca.colormap import linear
from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime
def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import time
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score,accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score, f1_score, recall_score, precision_score
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score
from itertools import combinations
center = (37.1623799231016, 127.05436890115905)
plt.style.use(['ggplot'])
rs = 123
np.random.seed(rs)
random.seed(rs)

In [11]:
df1 = pd.read_csv('C:/Users/simjh96/OneDrive/문서/Car_Accident/model/data/feature25.csv')

In [12]:
learn_cols = ["lon",
              "lat",
              "overspeed_cam_count100",
              "floating_pop_count50",
              "shortest_bump_dist",
              "parking_count12.5",
              "parking_count25",
              "car_count1000", 
              "child_count1000",
              "elem_kinder_count400",
              "numberSchoolZone_count400",
              "num_cram_school_count400",
              "shortest_cross_dist",
              "shortest_traffic_signal_dist",
              "shortest_sidewalk_dist",
              "barrier_nearby_count12.5",
              "barrier_nearby_count25",
              "chaos1_nearby_count12.5",
              "chaos1_nearby_count25",
              "width_nearby_count12.5",
              "width_nearby_count25",
              "cross_road_nearby_count12.5",
              "cross_road_nearby_count25"]

clust_cols = ["floating_pop_count50",
        "parking_count25",
        "car_count1000", 
        "child_count1000",
        "elem_kinder_count400",
        "numberSchoolZone_count400",
        "chaos1_nearby_count25",
        "cross_road_nearby_count25",
        "num_cram_school_count400",
        "shortest_cross_dist"]

In [13]:
#df1 feature 조절
#groupby로 count 하고
#등간이라고 생각되는 만큼 조절
df1.elem_kinder_count400.loc[df1.elem_kinder_count400 == 0] = 0
df1.elem_kinder_count400.loc[(df1.elem_kinder_count400 > 0)&(df1.elem_kinder_count400 <= 10)] = 1
df1.elem_kinder_count400.loc[(df1.elem_kinder_count400 > 10)] = 2

df1.numberSchoolZone_count400.loc[(df1.numberSchoolZone_count400 ==0 )] = 0
df1.numberSchoolZone_count400.loc[(df1.numberSchoolZone_count400 > 0)&(df1.numberSchoolZone_count400 <= 2)] = 1
df1.numberSchoolZone_count400.loc[(df1.numberSchoolZone_count400 > 2)] = 2

df1.num_cram_school_count400.loc[(df1.num_cram_school_count400 ==0 )] = 0
df1.num_cram_school_count400.loc[(df1.num_cram_school_count400 > 0)&(df1.num_cram_school_count400 <= 31)] = 1
df1.num_cram_school_count400.loc[(df1.num_cram_school_count400 > 31)] = 2

df1["accident_count12.5"] = (df1["accident_count12.5"] > 0).astype(float)

df1['is_bump20'] = df1['shortest_bump_dist'] < 20
df1['is_sidewalk12.5'] = df1['shortest_sidewalk_dist'] < 12.5

In [14]:
df1.loc[:,:] = MinMaxScaler().fit_transform(df1)

x25 = df1.loc[:,learn_cols]
y25 = df1["accident_count12.5"]

x25_train, x25_test, y25_train, y25_test = train_test_split(x25, y25, test_size=0.33, random_state=3)

df2 = df1.loc[:,clust_cols]

In [15]:
#cluster 중앙값 구하는 
def centre(cluster_id):
  currentMinimum = 99999
  datas = df2.loc[labels_s == cluster_id,:].to_numpy()
  
  for point_idx in range(len(datas)):
      distance_sum = 0
      for second_point_idx in range(len(datas)):
          if point_idx == second_point_idx : continue
          distance_sum += np.linalg.norm(datas[point_idx] - datas[second_point_idx])
      # print('>>>>>', point_idx, distance_sum) 

      if distance_sum < currentMinimum :
          currentMinimum = distance_sum 
          centre_point = datas[point_idx]
  return centre_point

#클러스터 끼리 중앙값중 가장 짧은 클러스터 구하기
def closest(cluster_id, ban_list):
  currentMinimum = 99999
  clusters = list(set(labels_s))
  centre_point = centre(cluster_id)

  distance = 0
  for second_point_idx in range(len(clusters)):
    if cluster_id == clusters[second_point_idx] or (clusters[second_point_idx] in ban_list): continue
    distance = np.linalg.norm(centre_point - centre(clusters[second_point_idx]))
    # print('>>>>>', second_point_idx, distance) 

    if distance < currentMinimum :
      currentMinimum = distance 
      closest_clust = clusters[second_point_idx]

  return closest_clust

In [16]:
#hyper param search
indexs = []
for min_s in range(2,20,4):
  for eps in np.linspace(0.01,1,20):
    indexs.append([eps, min_s])

animate = []
scores = []
n_clusters = []
n_noise = []
for min_s in range(2,20,4):
  for eps in np.linspace(0.01,1,20):
    db = DBSCAN(eps=eps, min_samples=min_s).fit(df2)
    labels = db.labels_

    df3 = pd.concat([df2,pd.DataFrame(db.labels_)],axis=1)
    animate.append(df3.groupby(0).count().floating_pop_count50)
    try:
      scores.append(metrics.silhouette_score(df2, db.labels_))
    except:
      scores.append(-1)
    n_clusters.append(len(set(labels)) - (1 if -1 in labels else 0))
    n_noise.append(list(labels).count(-1))

def init():
    line.set_data([], [])
    return (line,)

def animate_f(i):
    x = animate[i].index
    y = animate[i]
    line.set_data(x, y)
    ax.set_title('eps, min_samples = {}\n Silhouette Coefficient: {}'.format(indexs[i],scores[i]))
    return (line,)

fig, ax = plt.subplots()

ax.set_xlim(( -2, 300))
ax.set_ylim((0, 5000))

line, = ax.plot([], [], lw=2)

# anim = animation.FuncAnimation(fig, animate_f, init_func=init,
#                                frames=len(indexs), interval=len(indexs), blit=True)

# HTML(anim.to_html5_video())


KeyboardInterrupt: 

In [ ]:
#동질성이 보장될 클러스터 갯수 중 실루엣 가장 큰값
p = pd.concat([pd.DataFrame(scores),pd.DataFrame(n_clusters),pd.DataFrame(n_noise),pd.DataFrame(indexs)],axis=1)
p.columns = ['scores','n_clusters','n_noise','eps','min_sample']
p.sort_values(['scores']).iloc[-60:,:]

In [ ]:
#클러스터링 후 모든 클러스터 각각에 logistic 진행

db = DBSCAN(eps=0.088421, min_samples=2).fit(df2)
labels_s = db.labels_

x25_train_l = pd.concat([df2,pd.DataFrame(labels_s)],axis=1).loc[x25_train.index,:].sort_index()
x25_test_l = pd.concat([df2,pd.DataFrame(labels_s)],axis=1).loc[x25_test.index,:].sort_index()

test_predict_set = []
x25_test0_indexs = []

ban_list = list(set(x25_test_l.loc[:,0]) - set(x25_train_l.loc[:,0]))

for i in list(set(labels_s)):
  # 2nd clust == 0 인거 해보자
  if i in ban_list:
    closest_clust = closest(i,ban_list)
    print(i,"->",closest_clust)
    i = closest_clust

  x25_train0 = x25_train[x25_train_l.loc[:,0] == i]
  y25_train0 = y25_train[x25_train_l.loc[:,0] == i]
  x25_test0 = x25_test[x25_test_l.loc[:,0] == i]
  y25_test0 = y25_test[x25_test_l.loc[:,0] == i]

  if sum(x25_test_l.loc[:,0] == i) == 0: continue
  x25_test0_indexs.append(x25_test_l.loc[x25_test_l.loc[:,0] == i].index)

  model = sm.Logit(y25_train0,sm.add_constant(x25_train0,has_constant = "add"))  
  results = model.fit(method= 'lbfgs',maxiter =1000)
  predictions = results.predict(sm.add_constant(x25_test0,has_constant = "add"))
  print(i,'번째 F1 score',f1_score(y25_test0.astype(int),(predictions > 0.5).astype(int)))
  print(" ")

  test_predict_set.append([y25_test0,(predictions > 0.5).astype(int)])

In [ ]:
#전체 y25_test(2224개)에 대한 prediction 값 concat
y = pd.DataFrame(test_predict_set[0][0])
pr = pd.DataFrame(test_predict_set[0][1])
for i in range(len(test_predict_set)):
  if i == 0:continue
  y = pd.concat((y,pd.DataFrame(test_predict_set[i][0])))
  pr = pd.concat((pr,pd.DataFrame(test_predict_set[i][1])))


print(f1_score(y.to_numpy().reshape(len(y)),pr.to_numpy().reshape(len(pr))))

fig, (ax1,ax2) = plt.subplots(2,figsize=(40,10), sharex=True,sharey=True)
ax1.plot(y.to_numpy().reshape(len(y)))
ax2.plot(pr.to_numpy().reshape(len(pr)))